<a href="https://colab.research.google.com/github/GGMmattos/PIC/blob/main/Experimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importação das bibliotecas 

In [1]:
#Importing needed libraries 
!pip install -U scikit-learn

from pandas.core.window.ewm import OnlineExponentialMovingWindow
import pandas as pd #Biblioteca para carregamento de arquivo csv
import numpy as np
import seaborn as sns #Para visualização de gráficos
import matplotlib.pyplot as plt #Para visualização de gráficos
import plotly.express as px #Biblioteca para geração de gráfico dinámico
%matplotlib inline
from collections import Counter


#NLTK tools for text processing 

import re, nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


#Modeling packages 

from sklearn.model_selection import train_test_split
from sklearn.metrics._plot.confusion_matrix import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity ='all'


!pip install -U spacy
!python -m spacy download pt
import spacy



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 35.4 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.5.2
    Uninstalling spacy-3.5.2:
      Successfully uninstalled spacy-3.5.2
2023-05-31 19:59:53.232689: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-31 19:59:54.402748: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
⚠ As of spaCy v3.0, shortcuts like 'pt' are deprecated. Please use the
full pipeline package name 'pt_core_news_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Experimento com a HateBR 1 (sem pré-processamento)



In [3]:
from pandas.core.groupby import base
base_hatebr = pd.read_csv('/content/drive/MyDrive/Dataset/HateBR.csv')
base_hatebr

,instagram_comments,offensive_language,offensiveness_levels,hate_speech
0,este lixo ...,1,1,-1
1,Mais um lixo,1,1,-1
2,Essa nao tem vergonha na cara!!,1,2,-1
3,Essa mulher é doente.pilantra!,1,3,-1
4,Comunista safada...,1,2,"5,8"
...,...,...,...,...
6995,Time perdendo a credibilidade,0,0,0
6996,Siga em frente Presidente Bolsonaro! Afinal o ...,0,0,0
6997,Tantas coisas importantes para resolver e fica...,0,0,0
6998,"A TIME escolhe quem eles quiser, isso não sign...",0,0,0


In [4]:
base_hatebr.offensive_language.value_counts()

1    3500
0    3500
Name: offensive_language, dtype: int64

Dividindo em base de teste e base de treinamento



In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    base_hatebr.instagram_comments,
    base_hatebr.offensive_language, 
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.offensive_language
)

In [6]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (5600,)
Shape of X_test (1400,)


In [7]:
y_train.value_counts()

1    2800
0    2800
Name: offensive_language, dtype: int64

In [8]:
y_test.value_counts()

1    700
0    700
Name: offensive_language, dtype: int64

Treinamento com Naive Bayes




In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.85      0.81      0.83       700
           1       0.82      0.86      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



Support Vector Machine(SVM)

In [10]:
from sklearn.svm import SVC

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC( kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(kernel='linear'))])

              precision    recall  f1-score   support

           0       0.84      0.85      0.84       700
           1       0.85      0.84      0.84       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



Multilayer Perceptron(MLP) 

In [11]:
from sklearn.neural_network import MLPClassifier

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.83      0.85      0.84       700
           1       0.84      0.83      0.83       700

    accuracy                           0.84      1400
   macro avg       0.84      0.84      0.84      1400
weighted avg       0.84      0.84      0.84      1400



 Regressão Logística(LR)

In [12]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.83      0.82      0.83       700
           1       0.82      0.83      0.83       700

    accuracy                           0.83      1400
   macro avg       0.83      0.83      0.83      1400
weighted avg       0.83      0.83      0.83      1400



# Experimento com a HateBR 1 (com pre-processamento)



In [13]:
# comments = [] 
# stop_words = set(stopwords.words('portuguese')) #setando o banco de stopwords em PT-BR

# for words in base_hatebr ['instagram_comments']:
#   only_latters =re.sub("[^a-zA-Z]", " ", words) # somente letras
#   tokens = nltk.word_tokenize(only_latters) # tokenize the sentence 
#   lower_case = [l.lower() for l in tokens] # converter all letters to low case 
#   filtered_result = list(filter(lambda l: l not in stop_words, lower_case)) # Remove stopwords from the comments
#   comments.append(' '.join(filtered_result))
  
  # for i in range (len(comments)):
  #   words = nltk.word_tokenize(comments[i])
  #   newwords = [lemmatizer.lemmatize(word) for word in words]
  #   comments[i] = ' '.join(newwords)

Função de pré-processamento 



*   Foi notado que ao realizar o procedimento de deixar somente letras o modelo teve piora de pontuação F1, sem a função o texto continua com emojis e caracteres especiais...



In [14]:
npl = spacy.load("pt_core_news_sm")
def preprocess(text):
  comentarios = text.lower()
  # only_latters =re.sub("[^a-zA-Z]", " ", comentarios) # somente letras 
  doc = npl(comentarios)
  filtered_result = []
  for token in doc:
    if token.is_stop or token.is_punct :
      continue
    filtered_result.append(token.lemma_)
  return " ".join(filtered_result)


Criando uma nova coluna com o texto com pré-processamento

In [15]:
# print(preprocess(base_hatebr['instagram_comments']))
base_hatebr['preprocessed_comments'] =  base_hatebr['instagram_comments'].apply(preprocess)


In [16]:
base_hatebr

,instagram_comments,offensive_language,offensiveness_levels,hate_speech,preprocessed_comments
0,este lixo ...,1,1,-1,lixo
1,Mais um lixo,1,1,-1,lixo
2,Essa nao tem vergonha na cara!!,1,2,-1,nao vergonha cara
3,Essa mulher é doente.pilantra!,1,3,-1,mulher doente.pilantra
4,Comunista safada...,1,2,"5,8",comunista safar
...,...,...,...,...,...
6995,Time perdendo a credibilidade,0,0,0,time perder credibilidade
6996,Siga em frente Presidente Bolsonaro! Afinal o ...,0,0,0,siga frente presidente bolsonaro afinal intere...
6997,Tantas coisas importantes para resolver e fica...,0,0,0,tanto coisa importante resolver ficar dar ênfa...
6998,"A TIME escolhe quem eles quiser, isso não sign...",0,0,0,time escolhe querer significar


In [17]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    base_hatebr.	preprocessed_comments,
    base_hatebr.offensive_language, 
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.offensive_language
)

In [18]:
print('Shape of X_train', X_train.shape)
print('Shape of X_test', X_test.shape )

Shape of X_train (5600,)
Shape of X_test (1400,)


Treinamento com Naive Bayes

In [19]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report


#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

           0       0.86      0.84      0.85       700
           1       0.84      0.86      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



In [20]:

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report #comparação entre os resultados 
accuracy_score(y_test, y_pred) #retorna a porcentagem dos acertos 
     

0.8492857142857143

Support Vector Machine(SVM)

In [41]:
from sklearn.svm import SVC

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('SVM', SVC(C=2.0, kernel = 'linear'))
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('SVM', SVC(C=2.0, kernel='linear'))])

              precision    recall  f1-score   support

           0       0.84      0.88      0.86       700
           1       0.87      0.83      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



Multilayer Perceptron(MLP)

In [22]:
from sklearn.neural_network import MLPClassifier

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('MLP', MLPClassifier())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('MLP', MLPClassifier())])

              precision    recall  f1-score   support

           0       0.82      0.88      0.85       700
           1       0.87      0.81      0.84       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



 Regressão Logística(LR)

In [23]:
from sklearn.linear_model import LogisticRegression

clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()),
    ('LR', LogisticRegression())
])

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('LR', LogisticRegression())])

              precision    recall  f1-score   support

           0       0.84      0.87      0.85       700
           1       0.87      0.83      0.85       700

    accuracy                           0.85      1400
   macro avg       0.85      0.85      0.85      1400
weighted avg       0.85      0.85      0.85      1400



TOLD


In [49]:
from pandas.core.groupby import base
base_ToLDBR = pd.read_csv('/content/drive/MyDrive/ToLD-BR.csv')
base_ToLDBR

,text,homophobia,obscene,insult,racism,misogyny,xenophobia
0,Meu nivel de amizade com isis é ela ter meu in...,0.0,0.0,2.0,0.0,0.0,0.0
1,"rt @user @user o cara adultera dados, que fora...",0.0,0.0,1.0,0.0,0.0,0.0
2,@user @user @user o cara só é simplesmente o m...,0.0,2.0,1.0,0.0,0.0,0.0
3,eu to chorando vei vsf e eu nem staneio izone ...,0.0,1.0,0.0,0.0,0.0,0.0
4,Eleitor do Bolsonaro é tão ignorante q não per...,0.0,1.0,2.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...
20995,@user faz favor vai carai,0.0,0.0,0.0,0.0,0.0,0.0
20996,só queria conhecer alguém que não conhece o he...,1.0,0.0,0.0,0.0,0.0,0.0
20997,"vcs militam na hora errada em cima de memes, p...",0.0,0.0,0.0,0.0,0.0,0.0
20998,@user porra any eu tava c dor de cabeca e fui ...,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
base_ToLDBR .homophobia.value_counts()

0.0    20656
1.0      168
2.0      102
3.0       74
Name: homophobia, dtype: int64

In [52]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    base_ToLDBR .text,
    base_hatebr.homophobia, 
    test_size = 0.2, #20% das amostras vão para o conjunto de dados de teste
    random_state = 2022,
    stratify = base_hatebr.homophobia
)

In [53]:
#Criação do objeto Pipeline
clf = Pipeline([
    ('vectorizer_tfidf', TfidfVectorizer()), #using the ngram_range parameter
    ('Multi NB', MultinomialNB())
])

clf.fit(X_train, y_train)

#Obtem a precisão de X_test e e armazena em y_pred
y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

Pipeline(steps=[('vectorizer_tfidf', TfidfVectorizer()),
                ('Multi NB', MultinomialNB())])

              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99      4131
         1.0       0.00      0.00      0.00        34
         2.0       0.00      0.00      0.00        20
         3.0       0.00      0.00      0.00        15

    accuracy                           0.98      4200
   macro avg       0.25      0.25      0.25      4200
weighted avg       0.97      0.98      0.98      4200



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
base_hatebr.obscene.value_counts()

0.0    14348
1.0     4249
2.0     1791
3.0      612
Name: obscene, dtype: int64

In [44]:
base_hatebr.insult.value_counts()

0.0    16615
1.0     2516
2.0     1352
3.0      517
Name: insult, dtype: int64

In [45]:
base_hatebr.racism.value_counts()

0.0    20862
1.0      105
2.0       27
3.0        6
Name: racism, dtype: int64

In [46]:
base_hatebr.misogyny.value_counts()

0.0    20537
1.0      330
2.0      104
3.0       29
Name: misogyny, dtype: int64

In [47]:
base_hatebr.xenophobia.value_counts()

0.0    20849
1.0      109
2.0       27
3.0       15
Name: xenophobia, dtype: int64